# シュミレータ作り
目標: 分かりやすいシュミレータをnumpyで作る。

In [27]:
import numpy as np
import math
import scipy.sparse as sp

素晴らしいシュミレータを見つけたのでこれを参考にして作っていこうと思う。  
[シュミレータを作りながら量子コンピュータの仕組みを理解する](https://qiita.com/ohtaman/items/e121a2b0b4525b85b54d)

In [58]:
class Qubits(object):
    def __init__(self, n_bits):
        """
        Arguments:
            n_bits (int): 量子ビット数
        """
        self.n_bits   = n_bits                                     # 量子ビット数
        self.n_states = 2**n_bits                                  # 行数
        self._amp     = np.zeros(self.n_states, dtype=np.complex)  # 絶対値の二乗をとると確率になる
        self._amp[0]  = 1                                          # 0番目の状態の確率を1とする

    def set_bits(self, bits):
        """
        Arguments:
            bits (list): 状態を表現するリスト ex).|0010> なら [0, 0, 1, 0]
        Returens:
            self (Qubits): 自分自身
        """
        # 例えば |0010> なら idx=2番目(0010番目)となる
        idx            = sum(bit<<i for i, bit in enumerate(bits[::-1]))   # 1の位置をindexにする ( bit<<i = bit*2**i )
        self._amp      = np.zeros(self.n_states, dtype=np.complex)         # 絶対値の二乗をとると確率になる
        self._amp[idx] = 1.0                                               # index番目の確率を1とする
        return self

    def measure(self):
        """
        量子ビットを観測する
        後で第kビット目だけを観測するメソッドも作ることにする

        Returens:
            bits (list): 観測された状態を表すリスト
        """
        p    = np.abs(self._amp)**2                              # self.amp を観測確率にする
        idx  = np.random.choice(range(len(self._amp)), p=p)      # 適当にindexを決める。pは合計値が1となるlist
        bits = [idx>>i & 1 for i in range(self.n_bits)]         # index番目だけ状態が1となる。その他は0 ( idx>>i = int(idx/2**i) )
        return bits[::-1]                                       # 観測された結果を返す

    def apply(self, *operators):
        """
        量子ビットに演算子を適用する

        Arguments:
            operators (list): 演算子のリスト(何個でも可)
        Returens:
            applied (Qubits): 演算子適用後の量子ビット
        """
        amp = self._amp
        for op in operators:
            amp = op(amp)                      # 演算子のもつ op.__call__ メソッドでかけ算
        applied = self.__class__(self.n_bits)  # 自分のクラスを呼び出している
        applied._amp = amp                     # 演算後の状態を記録する
        return applied                         # 演算後の量子ビットを返す ( applied.__str__ が呼び出される )

    def __str__(self):
        """
        print時に呼び出されるメソッド
        
        Returns:
            (amp)|0010>
        """
        return " + ".join(
            ("{}|{:0" + str(self.n_bits) + "b}>").format(amp, i)
            for i, amp in enumerate(self._amp) if amp
        )

In [33]:
# Tests
qubits = Qubits(5)  # 5量子ビットを定義
qubits.set_bits([0, 0, 0, 1, 0])  # 4量子ビット目のみ|1>で、他は|0>

print(len(qubits._amp))  
print(qubits._amp[int('00010', 2)])
print(qubits._amp.sum())
print(qubits)

# Test Mearure
print(qubits.measure())

32
(1+0j)
(1+0j)
(1+0j)|00010>
[0, 0, 0, 1, 0]


演算のベースクラス

In [34]:
import abc

class Operator(object):
    __metaclass__ = abc.ABCMeta

    def __init__(self, n_bits):
        self.n_bits = n_bits

    @abc.abstractproperty
    def matrix(self):
        pass

    def __call__(self, amp):
        return self.matrix.dot(amp)

    def __str__(self):
        return str(self._matrix.todense())

単位行列 Id

In [35]:
class Id(Operator):
    def __init__(self, n_bits):
        super(Id, self).__init__(n_bits)
        self._matrix = sp.eye(2**n_bits)

    @property
    def matrix(self):
        return self._matrix

ビット反転演算子X

In [36]:
class X(Operator):
    def __init__(self, n_bits, target):
        super(X, self).__init__(n_bits)
        self.target = target

        self._matrix = sp.dok_matrix((2**n_bits, 2**n_bits))
        for upper in range(2**target):  # target より上位ビットをすべてなめる
            for lower in range(2**(n_bits - 1 - target)):  # target より下位のビットを全てなめる
                idx0 = upper*2**(n_bits - target) + lower  # target = 0 のインデックス
                idx1 = idx0 + 2**(n_bits - 1 - target)  # target = 1 のインデックス
                self._matrix[idx0, idx1] = 1
                self._matrix[idx1, idx0] = 1

    @property
    def matrix(self):
        return self._matrix

In [37]:
# Test X 第一引数:ビット数　第二引数:標的ビット
x=X(1,0)
print(x)
x2=X(2,1)
print(x2)

[[0. 1.]
 [1. 0.]]
[[0. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


アダマールゲートH

In [38]:
class H(Operator):
    def __init__(self, n_bits, target):
        super(H, self).__init__(n_bits)
        self.target = target

        self._matrix = sp.dok_matrix((2**n_bits, 2**n_bits))
        for upper in range(2**target):
            for lower in range(2**(n_bits - 1 - target)):
                idx0 = upper*2**(n_bits - target) + lower
                idx1 = idx0 + 2**(n_bits - 1 - target)
                self._matrix[idx0, idx0] = 1/np.sqrt(2)
                self._matrix[idx0, idx1] = 1/np.sqrt(2)
                self._matrix[idx1, idx0] = 1/np.sqrt(2)
                self._matrix[idx1, idx1] = -1/np.sqrt(2)

    @property
    def matrix(self):
        return self._matrix

In [39]:
# Test H
h=H(1,0)
print(h)

[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]


CNOT演算子

In [40]:
class CNot(Operator):
    def __init__(self, n_bits, control, target):
        super(CNot, self).__init__(n_bits)
        self.control = control
        self.target = target

        self._matrix = sp.dok_matrix((2**n_bits, 2**n_bits))
        for upper in range(2**target):
            for lower in range(2**(n_bits - 1 - target)):
                idx0 = upper*2**(n_bits - target) + lower
                idx1 = idx0 + 2**(n_bits - 1 - target)
                if self._get_control_bit(idx0):
                    self._matrix[idx0, idx1] = 1
                    self._matrix[idx1, idx0] = 1
                else:
                    self._matrix[idx0, idx0] = 1
                    self._matrix[idx1, idx1] = 1

    @property
    def matrix(self):
        return self._matrix

    def _get_control_bit(self, bits):
        return 1 & (bits >> (self.n_bits - 1 - self.control))

In [41]:
# Test CNOT(ビット数,制御,標的)　
cnot=CNot(2,0,1)
print(cnot)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


In [59]:
qubits = Qubits(2)
qubits.set_bits([1, 0])

# Test X 第一引数:ビット数　第二引数:標的ビット
x=X(2,0)
h=H(2,0)
print(x)
print(h)
print(qubits.apply(x,h))

[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]]
[[ 0.70710678  0.          0.70710678  0.        ]
 [ 0.          0.70710678  0.          0.70710678]
 [ 0.70710678  0.         -0.70710678  0.        ]
 [ 0.          0.70710678  0.         -0.70710678]]
(0.7071067811865475+0j)|00> + (0.7071067811865475+0j)|10>
